In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report

from evaluator_ANAELE import *

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

2024-03-11 21:12:42.414580: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 21:12:42.446719: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 21:12:43.026701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# FUNCTIONS

# to show performance

def evaluate(Y_pred,Y,S,will_print=1):
    '''returns model accuracy, final score, macro fscore ans TPR gap
    input : 2 np arrays of same dimension
    output : array of 4 values
    '''
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

    if will_print==1:
        #print results
        print('final score',final_score)
        print('macro_fscore',eval_scores['macro_fscore'])
        print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, eval_scores['macro_fscore'],1-eval_scores['TPR_GAP'] , eval_scores , confusion_matrices_eval

# to predict X_test and save to file

def save_X_test_true(X, model,name):
    Y_pred = model.predict(X)
    results=pd.DataFrame(y_pred, columns= ['score'])
    file_name = "Data_Challenge_MDI_341_"+str(name)+".csv"
    results.to_csv(file_name, header = None, index = None)
    
    return Y_pred

In [3]:
##############################################################
# LOAD DATA, 
#############################################################

# Load pickle file and convert to numpy array
with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)
 
#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

#create a label to distiguish 56 labels Y x 2 (man or woman)
# 0 to 27 = non sensitive group | 28 + [0 , 27] = 28 to 55 = sensitive group
Y56 = Y+28*S

X_test_true = dat['X_test']
S_test_true = dat['S_test']

# check size
print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)


dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [35]:
##############################################################
# train_test_split (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(X, Y56, test_size=0.2, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

##############################################################
# 1. Transform DataFrames into Tensors
##############################################################

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
S_train_tensor = torch.tensor(S_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
S_test_tensor = torch.tensor(S_test.values, dtype=torch.long)

Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=Y_train.nunique())
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=Y_train.nunique())

X_test_true_tensor = torch.tensor(X_test_true.values, dtype=torch.float32)

# impression des dimensions
print('train_tensor:',X_train_tensor.shape,Y_train_tensor.shape,S_train_tensor.shape, type(X_train_tensor))
print('test_tensor:',X_test_tensor.shape,Y_test_tensor.shape, S_test_tensor.shape, type(X_test_tensor))
print('Y_train_one_hot:',Y_train_one_hot.shape, type(Y_train_one_hot))
print('X_test_true_tensor:',X_test_true_tensor.shape, type(X_test_true_tensor))

train: (22199, 768) (22199,) (22199,)
test: (5550, 768) (5550,) (5550,)
train_tensor: torch.Size([22199, 768]) torch.Size([22199]) torch.Size([22199]) <class 'torch.Tensor'>
test_tensor: torch.Size([5550, 768]) torch.Size([5550]) torch.Size([5550]) <class 'torch.Tensor'>
Y_train_one_hot: torch.Size([22199, 28]) <class 'torch.Tensor'>
X_test_true_tensor: torch.Size([11893, 768]) <class 'torch.Tensor'>


In [18]:
##############################################################
#  DEFINE CUSTOM LOSS FUNCTION AND EVALUATION FUNCTIONS
#   
#   soft_f1_loss
#   macro_soft_f1_loss
#   calculate_exact_macro_f1
#   calculate_class_tpr_gap
#   average_tpr_gap_per_class
#   
##############################################################


def soft_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the F1 score as a loss function.
    """
    y_pred_probs = torch.softmax(y_pred, dim=1)
    tp = torch.sum(y_true * y_pred_probs, dim=0)
    pp = torch.sum(y_pred_probs, dim=0)
    ap = torch.sum(y_true, dim=0)
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    soft_f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
    loss = 1 - soft_f1.mean()  # Mean to aggregate over all classes
    return loss

def soft_macro_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the macro F1 score as a loss function.
    Calculates the F1 score for each class independently and then takes the average.
    """
    y_pred_probs = torch.softmax(y_pred, dim=1)
    tp = torch.sum(y_true * y_pred_probs, dim=0)
    pp = torch.sum(y_pred_probs, dim=0)
    ap = torch.sum(y_true, dim=0)
    
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    
    f1_per_class = 2 * (precision * recall) / (precision + recall + 1e-6)
    macro_f1 = torch.mean(f1_per_class)  # Average F1 score across all classes
    
    loss = 1 - macro_f1  # Minimizing loss is maximizing macro F1 score
    return loss


def get_macro_f1(y_true, y_pred):
    """
    Calculate the exact macro F1 score for evaluation.
    Input : any format as tensors will be converted to Tensors of true label if dim >1 . Can be :
        - Tensor of probabilities(y_pred_probs) dimension (n,28)
        - Tensor of labels, one hote encoded (y_pred_one_hot) dimension (n,28)
        - Tensor of labels (y_pred_tensor) dimension (n,1)
    Ouput : scalar
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    " predict macro f1"
    f1 = f1_score(y_true_labels.cpu().numpy(), y_pred_labels.cpu().numpy(), average='macro')
    return f1

def get_tpr_gap(y_true, y_pred, protected_attribute, class_idx):
    """
    Calculate the TPR gap for a specific class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred_probs: Tensor of predicted probabilities (after softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    - class_idx: Index of the class for which to calculate the TPR gap.
    
    Returns:
    - TPR gap for the specified class.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Calculate overall TPR for the current class
    overall_mask = y_true_labels == class_idx
    overall_tpr = torch.sum((y_pred_labels == class_idx) & overall_mask).float() / (torch.sum(overall_mask).float() + 1e-6)
    
    # Initialize list to store TPR for each protected group
    group_tprs = []
    
    # Calculate TPR for each protected group
    for group_val in protected_attribute.unique():
        group_mask = (protected_attribute == group_val) & overall_mask
        group_tpr = torch.sum((y_pred_labels == class_idx) & group_mask).float() / (torch.sum(group_mask).float() + 1e-6)
        group_tprs.append(group_tpr)
    
    # Calculate TPR gap for the current class
    tpr_gaps = torch.abs(torch.tensor(group_tprs) - overall_tpr)
    
    return torch.mean(tpr_gaps)  # Return the mean TPR gap for this class

def get_macro_tpr_gap(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap per class by calling tpr_gap for each class.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Initialize list to store TPR gaps for all classes
    class_tpr_gaps = []
    
    # Iterate over each class
    num_classes = len(y_true_labels.unique())
    for class_idx in range(num_classes):
        class_tpr_gap = get_tpr_gap(y_true_labels, y_pred_labels, protected_attribute, class_idx)
        class_tpr_gaps.append(class_tpr_gap)
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(class_tpr_gaps))
    
    return avg_tpr_gap


'''def get_tpr_gap(y_true, y_pred, protected_attribute, class_idx):
    """
    Calculate the TPR gap for a specific class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred_probs: Tensor of predicted probabilities (after softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    - class_idx: Index of the class for which to calculate the TPR gap.
    
    Returns:
    - TPR gap for the specified class.
    """
    # Convert one-hot labels to class indices for gathering
    y_true_indices = torch.argmax(y_true, dim=1)
    
    # Calculate overall TPR for the current class
    overall_mask = y_true_indices == class_idx
    overall_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & overall_mask).float() / (torch.sum(overall_mask).float() + 1e-6)
    
    # Initialize list to store TPR for each protected group
    group_tprs = []
    
    # Calculate TPR for each protected group
    for group_val in protected_attribute.unique():
        group_mask = (protected_attribute == group_val) & overall_mask
        group_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & group_mask).float() / (torch.sum(group_mask).float() + 1e-6)
        group_tprs.append(group_tpr)
    
    # Calculate TPR gap for the current class
    tpr_gaps = torch.abs(torch.tensor(group_tprs) - overall_tpr)
    
    return torch.mean(tpr_gaps)  # Return the mean TPR gap for this class
'''
'''def get_macro_tpr_gap(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap per class by calling tpr_gap for each class.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    # Apply softmax to get probabilities
    y_pred_probs = torch.softmax(y_pred, dim=1)
    
    # Initialize list to store TPR gaps for all classes
    class_tpr_gaps = []
    
    # Iterate over each class
    num_classes = len(y_true.unique())
    for class_idx in range(num_classes):
        class_tpr_gap = get_tpr_gap(y_true, y_pred_probs, protected_attribute, class_idx)
        class_tpr_gaps.append(class_tpr_gap)
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(class_tpr_gaps))
    
    return avg_tpr_gap
'''
def soft_final_score_loss(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    soft_macro_f1 = soft_macro_f1_loss(y_true, y_pred)  # Calculate soft macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true, y_pred, protected_attribute)  # Calculate TPR gap
    
    soft_final_score = ( soft_macro_f1 + (1 - macro_tpr_gap) ) / 2
    return soft_final_score

def get_final_score(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    macro_f1 = get_macro_f1(y_true_labels, y_pred_labels)  # Calculate macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true_labels, y_pred_labels, protected_attribute)  # Calculate macro TPR gap
    
    final_score = (macro_f1 + (1 - macro_tpr_gap)) / 2
    return final_score


**NN with customized loss function (soft macro f1 score)**
---

In [6]:
# 1. Transform DataFrames into Tensors
# ------------------------------------

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
S_train_tensor = torch.tensor(S_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
S_test_tensor = torch.tensor(S_test.values, dtype=torch.long)

Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=Y_train.nunique())
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=Y_train.nunique())

# impression des dimensions
print('train_tensor:',X_train_tensor.shape,Y_train_tensor.shape,S_train_tensor.shape, type(X_train_tensor))
print('test_tensor:',X_test_tensor.shape,Y_test_tensor.shape, S_test_tensor.shape, type(X_test_tensor))
print('Y_train_one_hot',Y_train_one_hot.shape, type(Y_train_one_hot))
      
# 2. Define the model and optimizer
# ---------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

optimizer = optim.SGD(model.parameters(), lr=0.5)

# 3. Train the model with the custom loss function final_eval
# -----------------------------------------------------------

num_epochs = 30000  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs_train = model(X_train_tensor)
    # loss = soft_final_score_loss(Y_train_one_hot.float(), outputs_train, S_train_tensor)
    loss = soft_macro_f1_loss(Y_train_one_hot.float(), outputs_train)
    # loss = get_macro_tpr_gap(Y_train_one_hot.float(), outputs_train, S_train_tensor )

    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 1000 == 0:
        model.eval()
        with torch.no_grad():
            # Calculate metrics for training data
            outputs_train = model(X_train_tensor) # probabilities

            # Evaluate predictions on training data
            final_score_train = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
            macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
            inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
            # Calculate metrics for test data
            outputs_test = model(X_test_tensor)

            # Evaluate predictions on training data
            final_score_test = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
            macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
            inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)
            
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            # print(f'Epoch {epoch + 1}, Loss: {loss.item()},  macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}')# Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')


# 4. Make Predictions and Evaluate with final_score
# -------------------------------------------------
            
with torch.no_grad():
    model.eval()
    Y_pred_probs = model(X_test_tensor) # dim = 28
    Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
    Y_pred_one_hot = torch.nn.functional.one_hot(Y_pred_tensor, num_classes=28)  # dim = 28
 
    macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
    inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
    final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
    print(f'Final Evaluation Score: {final_score.item()} Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

train_tensor: torch.Size([22199, 768]) torch.Size([22199]) torch.Size([22199]) <class 'torch.Tensor'>
test_tensor: torch.Size([5550, 768]) torch.Size([5550]) torch.Size([5550]) <class 'torch.Tensor'>
Y_train_one_hot torch.Size([22199, 28]) <class 'torch.Tensor'>
Epoch 1000, Loss: 0.4065406918525696, Final Score Train: 0.6408404111862183, Final Score Test: 0.6703691482543945, macro F1 Train: 0.6432239436677031, macro F1 Test: 0.5888075037853692, 1-TPR Gap Train: 0.9249047040939331, 1-TPR Gap Test: 0.9295457601547241
Epoch 2000, Loss: 0.3036802411079407, Final Score Train: 0.6094870567321777, Final Score Test: 0.6459752321243286, macro F1 Train: 0.722747985749416, macro F1 Test: 0.6317000182358751, 1-TPR Gap Train: 0.94172203540802, 1-TPR Gap Test: 0.9236504435539246
Epoch 3000, Loss: 0.27086883783340454, Final Score Train: 0.5999405384063721, Final Score Test: 0.6457803249359131, macro F1 Train: 0.7485006381197558, macro F1 Test: 0.6332508399781932, 1-TPR Gap Train: 0.948381781578064, 1

NameError: name 'final_score' is not defined

In [8]:
with torch.no_grad():
    model.eval()
    Y_pred_probs = model(X_test_tensor) # dim = 28
    Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
    Y_pred_one_hot = torch.nn.functional.one_hot(Y_pred_tensor, num_classes=28)  # dim = 28
 
    macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
    inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
    final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
    print(f'Final Evaluation Score: {final_score.item()} Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

Final Evaluation Score: 0.6483052372932434 Macro F1: 0.6330833983559353 1-TPR_gap: 0.929693877696991


In [9]:
# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred_tensor.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))

              precision    recall  f1-score   support

           0       0.83      0.54      0.66        81
           1       0.69      0.63      0.66       127
           2       0.79      0.89      0.84       458
           3       0.75      0.25      0.38        36
           4       0.79      0.69      0.73        48
           5       0.82      0.81      0.81        72
           6       0.89      0.76      0.82       178
           7       0.81      0.70      0.75        54
           8       0.82      0.50      0.62        18
           9       0.80      0.77      0.78        91
          10       0.60      0.41      0.49        22
          11       0.66      0.76      0.71       286
          12       0.85      0.72      0.78       110
          13       0.76      0.77      0.76       258
          14       0.82      0.67      0.74       112
          15       0.00      0.00      0.00        19
          16       0.48      0.42      0.45        33
          17       0.71    

/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
save_X_test_true(X_test_true, model,'custom_soft_f1_loss')

AttributeError: 'Sequential' object has no attribute 'predict'

**MODEL CUSTOM LOSS = FINAL SCORE**
---

In [27]:
    
# 2. Define the model and optimizer
# ---------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

optimizer = optim.SGD(model.parameters(), lr=0.1)

# 3. Train the model with the custom loss function final_eval
# -----------------------------------------------------------

num_epochs = 1000  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs_train = model(X_train_tensor)
    
    #loss = soft_final_score_loss(Y_train_one_hot.float(), outputs_train, S_train_tensor)
    loss = soft_macro_f1_loss(Y_train_one_hot.float(), outputs_train)*10 + get_macro_tpr_gap(Y_train_one_hot.float(), outputs_train, S_train_tensor )

    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        model.eval()
        with torch.no_grad():
            # Calculate metrics for training data
            outputs_train = model(X_train_tensor) # probabilities

            # Evaluate predictions on training data
            final_score_train = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
            macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
            inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
            # Calculate metrics for test data
            outputs_test = model(X_test_tensor)

            # Evaluate predictions on training data
            final_score_test = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
            macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
            inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)
            
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            # print(f'Epoch {epoch + 1}, Loss: {loss.item()},  macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}')# Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')


# 4. Make Predictions and Evaluate with final_score
# -------------------------------------------------
            
with torch.no_grad():
    model.eval()
    Y_pred_probs = model(X_test_tensor) # dim = 28
    Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
    Y_pred_one_hot = torch.nn.functional.one_hot(Y_pred_tensor, num_classes=28)  # dim = 28
 
    macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
    inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
    final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
    print(f'Final Evaluation Score: {final_score.item()} Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

train_tensor: torch.Size([22199, 768]) torch.Size([22199]) torch.Size([22199]) <class 'torch.Tensor'>
test_tensor: torch.Size([5550, 768]) torch.Size([5550]) torch.Size([5550]) <class 'torch.Tensor'>
Y_train_one_hot torch.Size([22199, 28]) <class 'torch.Tensor'>
Epoch 100, Loss: 9.229928970336914, Final Score Train: 0.5473254323005676, Final Score Test: 0.5484995245933533, macro F1 Train: 0.10194340859892699, macro F1 Test: 0.10541046804592533, 1-TPR Gap Train: 0.9927074313163757, 1-TPR Gap Test: 0.9915885925292969
Epoch 200, Loss: 8.115486145019531, Final Score Train: 0.6071764826774597, Final Score Test: 0.6037586331367493, macro F1 Train: 0.2593545053976912, macro F1 Test: 0.2511065513502886, 1-TPR Gap Train: 0.9549984335899353, 1-TPR Gap Test: 0.9564107060432434
Epoch 300, Loss: 6.555143356323242, Final Score Train: 0.6743472218513489, Final Score Test: 0.6614153981208801, macro F1 Train: 0.4129127061666681, macro F1 Test: 0.39077676498786784, 1-TPR Gap Train: 0.935781717300415, 1-

In [29]:
# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred_tensor.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))

              precision    recall  f1-score   support

           0       0.66      0.62      0.64        81
           1       0.70      0.57      0.63       127
           2       0.81      0.87      0.84       458
           3       0.00      0.00      0.00        36
           4       0.89      0.65      0.75        48
           5       0.81      0.78      0.79        72
           6       0.84      0.71      0.77       178
           7       0.75      0.74      0.75        54
           8       0.92      0.67      0.77        18
           9       0.79      0.81      0.80        91
          10       0.77      0.45      0.57        22
          11       0.55      0.83      0.66       286
          12       0.86      0.70      0.77       110
          13       0.74      0.73      0.73       258
          14       0.86      0.70      0.77       112
          15       0.00      0.00      0.00        19
          16       0.48      0.48      0.48        33
          17       0.74    

/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
Y_pred_probs = model(X_test_true_tensor)
Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)

results=pd.DataFrame(Y_pred_tensor, columns= ['score'])
name = 'NN_with_custom_loss'
file_name = "Data_Challenge_MDI_341_"+str(name)+".csv"
results.to_csv(file_name, header = None, index = None)


**2. REGRESSION WITH CUSTOM LOSS macro F1**
---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np

# Assuming model, optimizer, X_train_tensor, Y_train_one_hot, X_test_tensor, Y_test are already defined

# Convert Y_test to one-hot encoding if it's not already one-hot encoded
# This is necessary for consistency in our loss function calculations
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.int64) if isinstance(Y_test, pd.Series) else torch.from_numpy(Y_test).long()
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=28)


# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

# Define an optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

num_epochs = 10000  # Example number of epochs

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero the gradients
    
    # Forward pass on the training data
    outputs_train = model(X_train_tensor)
    loss_train = macro_soft_f1_loss(Y_train_one_hot.float(), outputs_train)
    
    # Backward pass and optimize
    loss_train.backward()
    optimizer.step()
    
    # No gradient computation needed for evaluation
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        
        # Forward pass on the validation data
        outputs_test = model(X_test_tensor)
        
        # Calculate the exact macro F1 score for both training and validation data
        f1_train = calculate_exact_macro_f1(Y_train_one_hot.float(), outputs_train)
        f1_test = calculate_exact_macro_f1(Y_test_one_hot.float(), outputs_test)
        
        model.train()  # Set the model back to training mode
    
    # Print loss and F1 score
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_train.item():.4f}, macro F1 Train: {f1_train:.4f}, macro F1 Test: {f1_test:.4f}')

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import torch

# Assuming model is already trained and X_test is a DataFrame

# Convert X_test to a PyTorch tensor
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Make predictions
with torch.no_grad():  # We do not need gradient computation for prediction
    model.eval()  # Set the model to evaluation mode
    Y_pred_probs = model(X_test_tensor)
    Y_pred = torch.argmax(Y_pred_probs, dim=1)  # Get the class with the highest probability

# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))

# If you want to use the exact F1 score for evaluation, you can directly use it from sklearn.metrics
from sklearn.metrics import f1_score
print("Exact F1 Score (micro):", f1_score(Y_test, Y_pred_df['Predicted'],average = 'micro'))  # 'weighted' for multi-class
print("Exact F1 Score (macro):", f1_score(Y_test, Y_pred_df['Predicted'], average='macro'))  # 'weighted' for multi-class

# Returning Y_pred as a DataFrame makes sense for further analysis or submission
#return Y_pred_df

**CUSTON LOSS FUNCTION TRP GAP**
---

In [ ]:
import torch

def gap_TPR(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap for each class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    # Apply softmax to get probabilities
    y_pred_probs = torch.softmax(y_pred, dim=1)
    
    # Convert one-hot labels to class indices for gathering
    y_true_indices = torch.argmax(y_true, dim=1)
    
    # Initialize TPR storage
    tpr_gaps = []
    
    # Iterate over each class
    num_classes = y_true.shape[1]
    for class_idx in range(num_classes):
        # Calculate TPR for the current class across all groups
        tpr_list = []
        
        # Calculate overall TPR for the current class
        overall_mask = y_true_indices == class_idx
        overall_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & overall_mask).float() / torch.sum(overall_mask).float()
        
        # Calculate TPR for each protected group
        for group_val in protected_attribute.unique():
            group_mask = (protected_attribute == group_val) & overall_mask
            group_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & group_mask).float() / torch.sum(group_mask).float()
            tpr_list.append(group_tpr)
        
        # Calculate TPR gap for the current class and store it
        tpr_gaps.append(torch.abs(torch.tensor(tpr_list) - overall_tpr))
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(tpr_gaps))
    
    return avg_tpr_gap

In [ ]:
print(type(Y_test),Y_test.shape)
print(type(Y_pred_probs),Y_pred_probs.shape)
get_macro_tpr_gap(Y_test,Y_pred_probs,S_test)

<class 'pandas.core.series.Series'> (5550,)
<class 'torch.Tensor'> torch.Size([5550, 28])


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not Series